In [3]:
from library_dicom.dicom_processor.model.reader.Instance_RTSS import Instance_RTSS

from library_dicom.rtss_processor.model.MaskBuilder_RTSS import MaskBuilder_RTSS
from library_dicom.dicom_processor.model.Series import Series

import matplotlib.pyplot as plt
import numpy as np

### RTSTRUCT file -> Mask 

In [ ]:
rtss_path = 

In [ ]:
mask_builder_rtss = MaskBuilder_RTSS(rtss_path, CT_path)

In [ ]:
from time import process_time
t1_start = process_time()
mask_4D = mask_builder_rtss.rtss_to_4D_mask()
t1_stop = process_time() 
print("diff début fin :", t1_stop-t1_start)  

In [ ]:
mask_builder_rtss.number_of_roi

In [ ]:
plt.imshow(mask_4D[:,:,308,0]) #0 pour ROI 1
plt.show()
#3Dslicer <= 309 slice |  308 ième => Python 

### Mask -> RTSTRUCT file 


In [ ]:
from library_dicom.rtss_processor.model.RTSS_Writer import RTSS_Writer
import pydicom

In [ ]:
dataset = {}
for i in range(1,11):
    data = {}
    data['ROIName'] = str(i)
    data['ROIGenerationAlgorithm'] = 'SEMIAUTOMATIC'
    dataset[i] = data
print(dataset)    

In [ ]:
new_rtss = RTSS_Writer(mask_4D, CT_path, dataset)


In [ ]:
directory = r'C:\Users\wendy\Documents\STAGE\DicomRT\201802409 BARAT GALPIN ELIANE\103893708 Tepscan FDG\new_rtss'
filename = r'test2.dcm'

In [ ]:
new_rtss.save_file(filename, directory)

In [ ]:
### Test RTSTRUCT CT ROI to PET ROI 

In [4]:
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT

ct_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.458434.20201008132231363'
pet_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.30343.20201008132234488'

rt_path = '/media/oncopole/DD 2To/Jakob/Example Data/1.2.826.0.1.3417726.3.1622984591/1.2.826.0.1.3417726.3.714187.20201008132226221'

In [10]:
serie_pet_object = SeriesPT(pet_path)
serie_ct_object = SeriesCT(ct_path)
instance_pet = serie_pet_object.get_instances_ordered()
instance_ct = serie_ct_object.get_instances_ordered()

In [15]:
instance_pet[0].get_instance_tags()


{'PixelSpacing': ["0.3985997409", "0.3924781443"],
 'ImagePosition': ["-49.68300975", "-55.30265248", "-57.10406392"],
 'ImageOrientation': ["1.0", "0.0", "0.0", "0.0", "1.0", "0.0"],
 'RescaleSlope': "2.86798479606006",
 'RescaleIntercept': "0.0",
 'SOPInstanceUID': '1.2.826.0.1.3417726.3.298287.20201008132234519'}

In [16]:
print(serie_pet_object.get_size_matrix())

[256, 256, 159]


In [17]:
instance_ct[0].get_instance_tags()

{'PixelSpacing': ["0.209992", "0.209992"],
 'ImagePosition': ["-50.293084", "-50.293084", "-65.307512"],
 'ImageOrientation': ["1.0", "0.0", "0.0", "0.0", "1.0", "0.0"],
 'RescaleSlope': "1.0",
 'RescaleIntercept': "-999.9912109",
 'SOPInstanceUID': '1.2.826.0.1.3417726.3.825854.20201008132231394'}

In [18]:
serie_ct_object.get_size_matrix()

[480, 480, 623]